In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Input, Conv2D, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [26]:
# --- Dataset paths ---
DATA_DIR = r"C:\Users\abhis\Desktop\SET\MLPROJECT\dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
MODEL_PATH = 'backend/model/rcnn_model.h5'


In [27]:
# --- Data Augmentation ---
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)


In [28]:
train_data = data_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)


Found 6328 images belonging to 2 classes.


In [29]:
val_data = data_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)



Found 1581 images belonging to 2 classes.


In [30]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Conv2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

def build_rcnn():
    input_layer = Input(shape=(224, 224, 3))

    # Pretrained ResNet50 as feature extractor
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)
    x = base_model.output

    # Region Proposal Network (RPN)
    rpn = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    rpn = BatchNormalization()(rpn)  # Added Batch Norm

    rpn_cls = Conv2D(9, (1, 1), activation='sigmoid', name='rpn_cls')(rpn)
    rpn_reg = Conv2D(36, (1, 1), activation='linear', name='rpn_reg')(rpn)

    # ROI Pooling (simplified)
    roi_pool = GlobalAveragePooling2D()(rpn)
    roi_pool = Dropout(0.5)(roi_pool)  # Added Dropout

    # Classification head
    classifier = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(roi_pool)
    classifier = Dropout(0.5)(classifier)
    classifier = Dense(1, activation='sigmoid', name='class_output')(classifier)

    # BBox Regression head
    bbox_regressor = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(roi_pool)
    bbox_regressor = Dense(4, activation='linear', name='bbox_output')(bbox_regressor)

    model = Model(inputs=input_layer, outputs=[classifier, bbox_regressor])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss={
            'class_output': 'binary_crossentropy',
            'bbox_output': 'mse'
        },
        metrics={
            'class_output': 'accuracy',
            'bbox_output': 'mae'
        }
    )

    return model


In [31]:
# --- Early Stopping ---
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [32]:
# --- Train the Model ---
model = build_rcnn()
early_stopping = EarlyStopping(monitor='val_class_output_loss', patience=5, restore_best_weights=True)

from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_class_output_loss', factor=0.5, patience=3, verbose=1)


history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)


Epoch 1/20
198/198 [==============================] - 2935s 14s/step - loss: 0.8245 - class_output_loss: 0.3498 - bbox_output_loss: 0.2731 - class_output_accuracy: 0.8560 - bbox_output_mae: 0.4075 - val_loss: 2.0172 - val_class_output_loss: 1.0832 - val_bbox_output_loss: 0.7348 - val_class_output_accuracy: 0.6863 - val_bbox_output_mae: 0.6800
Epoch 2/20
198/198 [==============================] - 2851s 14s/step - loss: 0.5101 - class_output_loss: 0.1875 - bbox_output_loss: 0.1256 - class_output_accuracy: 0.9396 - bbox_output_mae: 0.2736 - val_loss: 1.8488 - val_class_output_loss: 1.2316 - val_bbox_output_loss: 0.4228 - val_class_output_accuracy: 0.6863 - val_bbox_output_mae: 0.4521
Epoch 3/20
198/198 [==============================] - 2839s 14s/step - loss: 0.4232 - class_output_loss: 0.1434 - bbox_output_loss: 0.0882 - class_output_accuracy: 0.9526 - bbox_output_mae: 0.2244 - val_loss: 1.5349 - val_class_output_loss: 1.0614 - val_bbox_output_loss: 0.2847 - val_class_output_accuracy: 0.

In [33]:
model.save('backend/model/rcnn_model_3rd.h5') 
print(f"Model saved at {MODEL_PATH}")

C:\Users\abhis\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved at backend/model/rcnn_model.h5
